In [1]:
import pandas as pd
from bs4 import BeautifulSoup as bs
import os
import requests
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import pymongo

In [2]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [3]:
db = client.MarsDB
collection = db.Mars

In [4]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389


[WDM] - Get LATEST driver version for 89.0.4389
[WDM] - Trying to download new driver from https://chromedriver.storage.googleapis.com/89.0.4389.23/chromedriver_win32.zip
[WDM] - Driver has been saved in cache [C:\Users\eweber\.wdm\drivers\chromedriver\win32\89.0.4389.23]


In [5]:
news_url = 'https://redplanetscience.com/'
browser.visit(news_url)
response = requests.get(news_url)

In [6]:
news_html = browser.html
soup = bs(news_html, 'html.parser')

In [7]:
#News Title
news_title = soup.find('div', class_='content_title').text
print(news_title)

NASA Wins Two Emmy Awards for Interactive Mission Coverage


In [8]:
#News Paragraph
news_p = soup.find('div', class_='article_teaser_body').text
print(news_p)

NASA-JPL's coverage of the Mars InSight landing earns one of the two wins, making this the NASA center's second Emmy.


In [9]:
post = {
    'title': news_title,
    'paragraph': news_p,
    'url': news_html
}
collection.insert_one(post)

In [10]:
#Photo
url='https://spaceimages-mars.com'
browser.visit(url)

In [11]:
browser.links.find_by_partial_text("FULL IMAGE").click()

In [12]:
pic_html = browser.html
soup = bs(pic_html,'html.parser')


In [13]:
pic_url = soup.find('img', class_='fancybox-image')['src']
print(url)
print(pic_url)

https://spaceimages-mars.com
image/featured/mars2.jpg


In [14]:
featured_image_url = url + pic_url
featured_image_url

'https://spaceimages-mars.comimage/featured/mars2.jpg'

In [15]:
post1= {'featured_image_url': featured_image_url}
collection.insert_one(post1)

In [16]:
#Facts
facts_url = 'https://galaxyfacts-mars.com'
browser.visit(facts_url)

In [17]:
facts_html = browser.html
soup = bs(facts_html, 'html.parser')
#mars_facts = soup.find('table', class_ = 'tbody')

In [18]:
mars_table = pd.read_html(facts_html)
mars_table

[                         0                1                2
 0  Mars - Earth Comparison             Mars            Earth
 1                Diameter:         6,779 km        12,742 km
 2                    Mass:  6.39 × 10^23 kg  5.97 × 10^24 kg
 3                   Moons:                2                1
 4       Distance from Sun:   227,943,824 km   149,598,262 km
 5          Length of Year:   687 Earth days      365.24 days
 6             Temperature:     -87 to -5 °C      -88 to 58°C,
                       0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
 3                Moons:          2 ( Phobos & Deimos )
 4       Orbit Distance:       227,943,824 km (1.38 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                   -87 to -5 °C
 7         First Record:              2nd millennium BC

In [19]:
mars_df = mars_table[0]
mars_df

,0,1,2
0,Mars - Earth Comparison,Mars,Earth
1,Diameter:,"6,779 km","12,742 km"
2,Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
3,Moons:,2,1
4,Distance from Sun:,"227,943,824 km","149,598,262 km"
5,Length of Year:,687 Earth days,365.24 days
6,Temperature:,-87 to -5 °C,-88 to 58°C


In [20]:
mars_df.columns = ["Categories", "Mars", "Earth"]
mars_df

,Categories,Mars,Earth
0,Mars - Earth Comparison,Mars,Earth
1,Diameter:,"6,779 km","12,742 km"
2,Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
3,Moons:,2,1
4,Distance from Sun:,"227,943,824 km","149,598,262 km"
5,Length of Year:,687 Earth days,365.24 days
6,Temperature:,-87 to -5 °C,-88 to 58°C


In [21]:
marshtml = (mars_df.to_html())

In [22]:
print(marshtml)

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>Categories</th>
      <th>Mars</th>
      <th>Earth</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>Mars - Earth Comparison</td>
      <td>Mars</td>
      <td>Earth</td>
    </tr>
    <tr>
      <th>1</th>
      <td>Diameter:</td>
      <td>6,779 km</td>
      <td>12,742 km</td>
    </tr>
    <tr>
      <th>2</th>
      <td>Mass:</td>
      <td>6.39 × 10^23 kg</td>
      <td>5.97 × 10^24 kg</td>
    </tr>
    <tr>
      <th>3</th>
      <td>Moons:</td>
      <td>2</td>
      <td>1</td>
    </tr>
    <tr>
      <th>4</th>
      <td>Distance from Sun:</td>
      <td>227,943,824 km</td>
      <td>149,598,262 km</td>
    </tr>
    <tr>
      <th>5</th>
      <td>Length of Year:</td>
      <td>687 Earth days</td>
      <td>365.24 days</td>
    </tr>
    <tr>
      <th>6</th>
      <td>Temperature:</td>
      <td>-87 to -5 °C</td>
      <td>-88 to 58°C</td>
   

In [23]:
mars_df.to_csv('../marsData.csv',index=False)

In [24]:
#Hemisphere
hemi_url = "https://marshemispheres.com/"
browser.visit(hemi_url)

In [25]:
hemi_html = browser.html
soup = bs(hemi_html, "html.parser")

In [26]:
hemi = soup.find_all("div", class_="item")
#hemi

In [27]:
hemi_info = []
for result in hemi:
    
    h_title = result.find("h3").text
    hemi_img = result.find("a", class_="itemLink product-item")["href"]
    
    browser.visit(hemi_url + hemi_img)
    
    image_html = browser.html
    web_info = bs(image_html, "html.parser")

    img_url = hemi_url + web_info.find("img", class_="wide-image")['src']
    
    hemi_info.append({"title":h_title, "img_url":img_url})
    browser.links.find_by_partial_text('Back').click()
    print(h_title)
    print(img_url)
    post2 = {
        'title': h_title,
        'img_url': img_url
    }
    collection.insert_one(post2)

Cerberus Hemisphere Enhanced
https://marshemispheres.com/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg
Schiaparelli Hemisphere Enhanced
https://marshemispheres.com/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg
Syrtis Major Hemisphere Enhanced
https://marshemispheres.com/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg
Valles Marineris Hemisphere Enhanced
https://marshemispheres.com/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg
